In [1]:
%matplotlib inline
import pandas as pd
import cv2
import numpy as np
from matplotlib import pyplot as plt

df = pd.read_csv("data/1105573_SELECT_t___FROM_data_data_t.csv",header=None,index_col=0)
df = df.rename(columns={0:"no", 1: "CAPTDATA", 2: "CAPTIMAGE",3: "timestamp"})

/home/cilab/.local/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [2]:
%%time
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1105573 entries, 1 to 1105588
Data columns (total 3 columns):
CAPTDATA     1105573 non-null object
CAPTIMAGE    1105573 non-null object
timestamp    1105573 non-null object
dtypes: object(3)
memory usage: 33.7+ MB
CPU times: user 207 ms, sys: 220 µs, total: 207 ms
Wall time: 204 ms


In [3]:
df.sample(5)

,CAPTDATA,CAPTIMAGE,timestamp
0,,,
482201,SbXgCCvsAhRod0graaUo3w==,0x89504E470D0A1A0A0000000D49484452000001180000...,2019-11-04 09:31:36
486109,lqjXKsMNfaIY1QvDp8zuSQ==,0x89504E470D0A1A0A0000000D49484452000001180000...,2019-11-04 09:40:00
24706,zejO4cpgQ+2rWl8xfV5D5Q==,0x89504E470D0A1A0A0000000D49484452000001180000...,2019-11-03 17:02:25
330897,JXB+V74q0EEpoL3GWN4Csg==,0x89504E470D0A1A0A0000000D49484452000001180000...,2019-11-04 04:04:09
488358,GOwDZjHTl7XKzazcX3G2KQ==,0x89504E470D0A1A0A0000000D49484452000001180000...,2019-11-04 09:44:56


In [4]:
def alpha_to_gray(img):
    alpha_channel = img[:, :, 3]
    _, mask = cv2.threshold(alpha_channel, 128, 255, cv2.THRESH_BINARY)  # binarize mask
    color = img[:, :, :3]
    img = cv2.bitwise_not(cv2.bitwise_not(color, mask=mask))
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [5]:
def preprocess(data):
    data = bytes.fromhex(data[2:])
    img = cv2.imdecode( np.asarray(bytearray(data), dtype=np.uint8), cv2.IMREAD_UNCHANGED )
    img = alpha_to_gray(img)
    kernel = np.ones((3, 3), np.uint8)
    img = cv2.dilate(img, kernel, iterations=1)
    img = cv2.medianBlur(img, 3)
    kernel = np.ones((4, 4), np.uint8)
    img = cv2.erode(img, kernel, iterations=1)
#     plt.imshow(img)
    return img

In [ ]:
df["IMAGE"] = df["CAPTIMAGE"].apply(preprocess)

In [ ]:
def bounding(gray):
#     data = bytes.fromhex(df["CAPTIMAGE"][1][2:])
#     image = cv2.imdecode( np.asarray(bytearray(data), dtype=np.uint8), cv2.IMREAD_UNCHANGED )

#     alpha_channel = image[:, :, 3]
#     _, mask = cv2.threshold(alpha_channel, 128, 255, cv2.THRESH_BINARY)  # binarize mask
#     color = image[:, :, :3]
#     src = cv2.bitwise_not(cv2.bitwise_not(color, mask=mask))

    ret, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    binary = cv2.bitwise_not(binary)

    contours, hierachy = cv2.findContours(binary, cv2.RETR_EXTERNAL , cv2.CHAIN_APPROX_NONE)

    ans = []
    for h, tcnt in enumerate(contours):
        x,y,w,h = cv2.boundingRect(tcnt)
        if h < 20:
            continue
        if 50 < w < 100: # 2개가 붙어 있는 경우
            ans.append([x,y,w//2+5,h])
            ans.append([x+(w//2)-5,y,w//2+5,h])
            continue
#         cv2.rectangle(src,(x,y),(x+w,y+h),(255,0,0),1)
        ans.append([x,y,w,h])
    return ans
    # cv2.destroyAllWindows()

In [ ]:
df["bounding"] = df["IMAGE"].apply(bounding)

In [ ]:
def draw_bounding(idx):
    CAPTIMAGE = df["CAPTIMAGE"][idx]
    bounding = df["bounding"][idx]
    
    data = bytes.fromhex(CAPTIMAGE[2:])
    image = cv2.imdecode( np.asarray(bytearray(data), dtype=np.uint8), cv2.IMREAD_UNCHANGED )

    alpha_channel = image[:, :, 3]
    _, mask = cv2.threshold(alpha_channel, 128, 255, cv2.THRESH_BINARY)  # binarize mask
    color = image[:, :, :3]
    src = cv2.bitwise_not(cv2.bitwise_not(color, mask=mask))

    for x,y,w,h in bounding:
#         print(x,y,w,h)
        cv2.rectangle(src,(x,y),(x+w,y+h),(255,0,0),1)
    return src

In [ ]:
import random

nrows = 4
ncols = 4
fig, axes = plt.subplots(nrows=nrows, ncols=ncols)
fig.set_size_inches((16, 6)) 
for i in range(nrows):
    for j in range(ncols):
        idx = random.randrange(20,22800)
        axes[i][j].set_title(str(idx))
        axes[i][j].imshow(draw_bounding(idx))
fig.tight_layout()
plt.savefig('sample.png')
plt.show()

In [ ]:
charImg = []
for idx in df.index:
    IMAGE = df["IMAGE"][idx]
    bounding = df["bounding"][idx]
    
    for x,y,w,h in bounding:
        newImg = IMAGE[y:y+h,x:x+w]
        newImg = cv2.resize(newImg, dsize=(41, 38), interpolation=cv2.INTER_NEAREST)
        charImg.append(newImg/255.0)


In [ ]:
# cast to numpy arrays
trainingImages = np.asarray(charImg)

# reshape img array to vector
def reshape_image(img):
    return np.reshape(img,len(img)*len(img[0]))

img_reshape = np.zeros((len(trainingImages),len(trainingImages[0])*len(trainingImages[0][0])))

for i in range(0,len(trainingImages)):
    img_reshape[i] = reshape_image(trainingImages[i])

In [ ]:
%%time
from sklearn.cluster import KMeans

import matplotlib.pyplot  as plt

import seaborn as sns



# create model and prediction

model = KMeans(n_clusters=300,algorithm='auto')

model.fit(img_reshape)

predict = pd.DataFrame(model.predict(img_reshape))

predict.columns=['predict']

In [ ]:
import pickle
pickle.dump(model, open("KMeans_300_1105573.pkl", "wb"))

In [ ]:
%%time
import random

r = pd.concat([pd.DataFrame(img_reshape),predict],axis=1)

nrows = 4
ncols = 10
fig, axes = plt.subplots(nrows=nrows, ncols=ncols)
fig.set_size_inches((16, 6))

for j in range(300):
    i = 0
    for idx in r[r["predict"] == j].sample(nrows * ncols).index:
        axes[i // ncols][i % ncols].set_title(str(idx))
        axes[i // ncols][i % ncols].imshow(trainingImages[idx])
        i+=1
    fig.tight_layout()
    plt.savefig('res_1105573/sample_' + str(j) + '.png')